# Embeddings Experiment 02 - Removal of Function End Tags

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

In [ ]:
# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
new_tokens = ['<FUNC1_STT>', '<FUNC2_STT>']

In [ ]:
# Add the new words to the tokenizer's vocabulary


tokenizer.add_tokens(new_tokens)

# Resize the GPT-2 model's embedding layer to accommodate the new tokens
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Print embeddings before training
print("Embeddings before training:")
new_token_embeddings = model.transformer.wte.weight[-2:]  # Get embeddings for the new tokens
print(new_token_embeddings)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

dataset_path = 'usage_dataset_2.txt'

def tokenize_sentence(sentence):
    tokens = tokenizer.encode(sentence, add_special_tokens=False)
    return tokens

def process_dataset(dataset):
    tokenized_dataset = []
    sentences = dataset.split('\n')
    for sentence in sentences:
        if sentence.strip() != '':
            tokens = tokenize_sentence(sentence)
            tokenized_dataset.append(tokens)
    return tokenized_dataset

# Read the dataset file
with open(dataset_path, 'r', encoding='utf-8') as file:
    dataset = file.read()

# Tokenize each sentence individually
tokenized_dataset = process_dataset(dataset)

# Pad the tokenized sequences
padded_dataset = pad_sequence([torch.tensor(tokens) for tokens in tokenized_dataset], batch_first=True)

# Convert tokenized dataset to PyTorch tensors
inputs = padded_dataset[:, :-1]  # Exclude the last token for prediction
labels = padded_dataset[:, 1:]   # Shift the input to generate labels

In [ ]:
# Convert tokenized dataset to PyTorch tensors
inputs = torch.tensor(padded_dataset[:-1]).unsqueeze(0)  # Exclude the last token for prediction
labels = torch.tensor(padded_dataset[1:]).unsqueeze(0)   # Shift the input to generate labels

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
# Set the model in training mode
model.to(device)
model.train()

# Fine-tune the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
for epoch in range(15):  # You can adjust the number of epochs as needed
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

In [ ]:
# Print embeddings after training
print("\nEmbeddings after training:")
new_token_embeddings = model.transformer.wte.weight[-2:]  # Get embeddings for the new tokens
print(new_token_embeddings)

In [ ]:
# Save the fine-tuned model
save_path = 'fine_tuned_model.pth'
torch.save(model.state_dict(), save_path)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between each pair of new embeddings
similarity_matrix = cosine_similarity(new_token_embeddings.cpu().detach().numpy())

# Print cosine similarity matrix
print("\nCosine Similarity Matrix:")
print(similarity_matrix)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between each pair of new embeddings
similarity_matrix = cosine_similarity(new_token_embeddings.cpu().detach().numpy())

# Print cosine similarity matrix
print("\nCosine Similarity Matrix:")
print(similarity_matrix)

In [ ]:
def generate_text(seed_text, max_length=50, temperature=0.8):
    input_ids = tokenizer.encode(seed_text, return_tensors='pt')
    input_ids = input_ids.to(device)

    attention_mask = torch.ones_like(input_ids)

    # Generate text
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        num_return_sequences=1
    )

    # Decode and return generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
generated_text = generate_text("I looked through the thesaurus and found that the synonym for pretty is")
print("Generated text:", generated_text)

In [ ]:
generated_text = generate_text("I found that the opposite of hate is")
print("Generated text:", generated_text)

In [ ]:
generated_text = generate_text("Yesterday, my dog and I walked to the")
print("Generated text:", generated_text)

In [ ]:
def find_similar_words(word, top_k=5):
    word_embedding = model.transformer.wte.weight[tokenizer.encode(word)[0]].detach().cpu().numpy()

    # Compute cosine similarity between word embedding and all other embeddings
    embeddings = model.transformer.wte.weight.detach().cpu().numpy()
    similarities = cosine_similarity([word_embedding], embeddings)[0]

    # Get indices of top-k similar words
    top_indices = similarities.argsort()[-top_k:][::-1]

    # Decode and return top-k similar words
    similar_words = [tokenizer.decode([index]) for index in top_indices]
    return similar_words

In [ ]:
# Find similar words based on embeddings

for token in new_tokens:
  similar_words = find_similar_words(token)
  print("\nSimilar words to", token + ":")
  for similar_word in similar_words:
      print(similar_word)
